In [ ]:
# Importar bibliotecas
import json
import pandas as pd

# Montar Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
file_path = '/content/drive/Shareddrives/CHEC/data_chec/datos/Data_Total/Clima/Manizales-2022-2023.json'

with open(file_path, 'r') as file:
    data = json.load(file)

# Inspeccionar las claves y estructuras
print(type(data))  # ¿Es un dict o una lista?
print(data.keys() if isinstance(data, dict) else data[0])  # Claves o primer elemento

<class 'dict'>
dict_keys(['city_id', 'city_name', 'country_code', 'data', 'lat', 'lon', 'sources', 'state_code', 'station_id', 'timezone'])


In [ ]:
data['lon']

-75.51738

In [ ]:
df = pd.DataFrame(data['data'])
df['city_name'] = data['city_name']
df['LATITUD'] = data['lat']
df['LONGITUD'] = data['lon']

In [ ]:
df

,app_temp,azimuth,clouds,datetime,dewpt,dhi,dni,elev_angle,ghi,h_angle,...,ts,uv,vis,weather,wind_dir,wind_gust_spd,wind_spd,city_name,LATITUD,LONGITUD
0,11.0,173.5,50,2022-12-03:05,9.6,0,0,-72.8,0,None,...,1670043600,0.0,9.0,"{'code': 803, 'icon': 'c03n', 'description': '...",165,1.32,1.32,Manizales,5.063,-75.51738
1,11.0,137.6,45,2022-12-03:06,9.6,0,0,-66.2,0,None,...,1670047200,0.0,9.0,"{'code': 803, 'icon': 'c03n', 'description': '...",155,1.02,1.02,Manizales,5.063,-75.51738
2,13.4,122.0,41,2022-12-03:07,10.2,0,0,-54.6,0,None,...,1670050800,0.0,9.0,"{'code': 500, 'icon': 'r01n', 'description': '...",140,1.70,1.69,Manizales,5.063,-75.51738
3,12.8,115.3,37,2022-12-03:08,10.0,0,0,-41.4,0,None,...,1670054400,0.0,9.0,"{'code': 500, 'icon': 'r01n', 'description': '...",130,3.00,3.00,Manizales,5.063,-75.51738
4,12.2,112.4,33,2022-12-03:09,9.6,0,0,-27.7,0,None,...,1670058000,0.0,9.0,"{'code': 500, 'icon': 'r01n', 'description': '...",115,2.86,2.86,Manizales,5.063,-75.51738
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,19.0,248.4,50,2023-12-03:00,11.7,0,0,-17.7,0,None,...,1701561600,0.0,9.0,"{'code': 803, 'icon': 'c03n', 'description': '...",110,3.20,2.14,Manizales,5.063,-75.51738
8756,18.4,247.1,50,2023-12-03:01,11.4,0,0,-31.5,0,None,...,1701565200,0.0,9.0,"{'code': 803, 'icon': 'c03n', 'description': '...",110,2.18,2.18,Manizales,5.063,-75.51738
8757,14.7,243.4,50,2023-12-03:02,11.1,0,0,-45.1,0,None,...,1701568800,0.0,9.0,"{'code': 803, 'icon': 'c03n', 'description': '...",110,2.20,2.16,Manizales,5.063,-75.51738
8758,12.9,235.0,50,2023-12-03:03,10.4,0,0,-58.0,0,None,...,1701572400,0.0,9.0,"{'code': 803, 'icon': 'c03n', 'description': '...",105,2.20,2.20,Manizales,5.063,-75.51738


In [ ]:
import folium
from folium import LinearColormap
from folium.plugins import MarkerCluster
from shapely.geometry import MultiPoint
from shapely.geometry.polygon import Polygon
import numpy as np
import random

# Crear un mapa base
m = folium.Map(location=[37.7749, -122.4194], zoom_start=13)

# Generar datos aleatorios cercanos a un punto central
def generate_random_points(center, num_points, radius=0.01):
    """Genera puntos aleatorios alrededor de un punto central dentro de un radio."""
    lat_center, lon_center = center
    return [
        [
            lat_center + random.uniform(-radius, radius),
            lon_center + random.uniform(-radius, radius),
        ]
        for _ in range(num_points)
    ]

# Centro para los puntos y número de puntos
centers = [
    [37.7749, -122.4194],
    [37.7849, -122.4294],
    [37.7649, -122.4094],
]
all_data = []
marker_clusters = []

for i, center in enumerate(centers):
    # Generar datos aleatorios para cada cluster
    data = generate_random_points(center, num_points=15)
    all_data.extend(data)

    # Crear un MarkerCluster para cada conjunto de puntos
    cluster = MarkerCluster(name=f"Cluster {i + 1}")
    for point in data:
        folium.Marker(location=point).add_to(cluster)
    marker_clusters.append(cluster)

    # Calcular el área del cluster y agregar al mapa
    if len(data) > 2:
        points = MultiPoint([tuple(loc) for loc in data])
        hull = points.convex_hull
        if isinstance(hull, Polygon):
            coordinates = np.array(hull.exterior.coords)
            '''folium.Polygon(
                locations=coordinates.tolist(),
                color="blue",
                fill=True,
                fill_color="blue",
                fill_opacity=0.3,
            ).add_to(m)'''

# Añadir los clusters al mapa
for cluster in marker_clusters:
    cluster.add_to(m)

# Crear una escala de colores para las temperaturas (de 15°C a 30°C)
temperature_scale = LinearColormap(
    colors=["blue", "green", "yellow", "red"],  # Colores para las temperaturas
    vmin=15,  # Mínima temperatura
    vmax=30,  # Máxima temperatura
)

# Añadir una leyenda de la escala de colores para la temperatura
temperature_scale.caption = "Temperatura (°C)"
temperature_scale.add_to(m)

In [ ]:
m

In [112]:
import folium
from folium import LinearColormap
from folium.plugins import MarkerCluster
from shapely.geometry import MultiPoint, LineString
from shapely.geometry.polygon import Polygon
from shapely.ops import unary_union
import numpy as np

# Crear un mapa base

# Crear el mapa base centrado en Manizales, Caldas, Colombia
m = folium.Map(
    location=[5.0689, -75.5179],  # Coordenadas de Manizales
    zoom_start=13,
    tiles='None',
    attr='Map tiles by Stamen Design, CC BY 3.0 — Map data © OpenStreetMap contributors'
)

# Agregar el estilo satelital de Esri
folium.TileLayer(
    tiles="https://server.arcgisonline.com/ArcGIS/rest/services/World_Topo_Map/MapServer/tile/{z}/{y}/{x}",
    attr="Esri",
    name="Esri Satellite",
    overlay=False,
    control=False  # Desactiva el control de capas para que solo muestre Esri
).add_to(m)

# Datos de ejemplo: lista de ubicaciones (coordenadas de Manizales, Caldas, Colombia)
data = [
    [5.0689, -75.5179],
    [5.0789, -75.5079],
    [5.0589, -75.5279],
    [5.0789, -75.5179],
    [5.0689, -75.5279],
]

# Lista de puntos (coordenadas de Manizales, Caldas, Colombia)
points = [
    [5.0689, -75.5179],
    [5.0789, -75.5079],
    [5.0589, -75.5279],
    [5.0789, -75.5179],
    [5.0689, -75.5279],
    [5.0489, -75.5379],
    [5.0389, -75.5479],
    [5.0189, -75.5579],
    [5.0589, -75.5679],
    [5.0789, -75.5779],
]

data_1 = [
    [5.0489, -75.5379],
    [5.0389, -75.5479],
    [5.0189, -75.5579],
    [5.0589, -75.5679],
    [5.0789, -75.5779],
]

# Crear un MarkerCluster
marker_cluster = MarkerCluster()

# Crear un MultiPoint de los puntos y calcular el Convex Hull
multi_point = MultiPoint([tuple(point) for point in points])
hull = multi_point.convex_hull

# Obtener las coordenadas del polígono del Convex Hull
hull_coords = list(hull.exterior.coords)

# Agregar el polígono del Hull al mapa
folium.Polygon(
    locations=np.array(hull_coords).tolist(),
    color="red",
    weight=0,
    fill=True,
    fill_color="orange",
    fill_opacity=0.2,
).add_to(m)

# Agregar los puntos al mapa como CircleMarker
for point in points:
    folium.CircleMarker(
        location=point,
        radius=2,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=1.0,
        popup="Lat: {} | Lng: {}".format(point[0], point[1])
    ).add_to(marker_cluster)

# Crear una LineString con los puntos para formar la "camino"
line = LineString(data)

# Crear un buffer alrededor de la LineString para obtener una especie de área
buffer = line.buffer(0.0005)  # Ajustar el valor según el tamaño deseado del área

# Convertir el buffer a un polígono y agregarlo al mapa
if isinstance(buffer, Polygon):
    coordinates = np.array(buffer.exterior.coords)
    folium.Polygon(
        locations=coordinates.tolist(),
        color="green",
        weight=0,
        fill=True,
        fill_color="green",
        fill_opacity=0.3,
        popup="Buffer area around markers"
    ).add_to(m)

# Crear una LineString con los puntos para formar la "camino"
line = LineString(data_1)

# Crear un buffer alrededor de la LineString para obtener una especie de área
buffer = line.buffer(0.0005)  # Ajustar el valor según el tamaño deseado del área

# Convertir el buffer a un polígono y agregarlo al mapa
if isinstance(buffer, Polygon):
    coordinates = np.array(buffer.exterior.coords)
    folium.Polygon(
        locations=coordinates.tolist(),
        color="green",
        weight=0,
        fill=True,
        fill_color="green",
        fill_opacity=0.3,
        popup="Buffer area around markers"
    ).add_to(m)

# Añadir MarkerCluster al mapa
marker_cluster.add_to(m)


# Crear una escala de colores para las temperaturas (de 15°C a 30°C)
temperature_scale = LinearColormap(
    colors=["blue", "green", "yellow", "red"],  # Colores para las temperaturas
    vmin=15,  # Mínima temperatura
    vmax=30,  # Máxima temperatura
)

# Añadir una leyenda de la escala de colores para la temperatura
temperature_scale.caption = "Temperatura (°C)"
temperature_scale.add_to(m)

In [113]:
m

In [89]:
import folium
from folium.plugins import MarkerCluster
from shapely.geometry import MultiPoint, LineString
from shapely.geometry.polygon import Polygon
from shapely.ops import unary_union
import numpy as np

# Método 1: Convex Hull (envolvente convexa)
def crear_poligono_convex_hull(points):
    """Crea un polígono usando Convex Hull"""
    multi_point = MultiPoint(points)
    polygon = multi_point.convex_hull
    return polygon

# Método 2: Buffer de puntos
def crear_poligono_buffer(points, buffer_distance=0.0005):
    """Crea un polígono con buffer alrededor de los puntos"""
    multi_point = MultiPoint(points)
    polygon = multi_point.buffer(buffer_distance)
    return polygon.simplify(tolerance=0.0001)

# Método 3: Polígono directo desde vértices
def crear_poligono_vertices(points):
    """Crea un polígono directamente de los vértices"""
    return Polygon(points)

def generar_mapa_con_poligonos(points_list):
    """Genera un mapa con múltiples polígonos"""
    m = folium.Map(location=[37.7749, -122.4194], zoom_start=13)

    # Colores para diferentes métodos de creación
    colores = ['blue', 'green', 'red']

    for i, points in enumerate(points_list):
        # Crear cluster de marcadores
        marker_cluster = MarkerCluster().add_to(m)

        # Agregar puntos como marcadores
        for point in points:
            folium.CircleMarker(
                location=point,
                radius=2,
                color='black',
                fill=True,
                fill_color='black',
                fill_opacity=1.0
            ).add_to(marker_cluster)

        # Crear polígono con diferentes métodos
        poligonos = [
            crear_poligono_convex_hull(points),
            crear_poligono_buffer(points),
            crear_poligono_vertices(points)
        ]

        # Agregar polígonos al mapa
        for j, polygon in enumerate(poligonos):
            if isinstance(polygon, Polygon):
                coordinates = np.array(polygon.exterior.coords)
                folium.Polygon(
                    locations=coordinates.tolist(),
                    color=colores[j],
                    weight=2,
                    fill=True,
                    fill_color=colores[j],
                    fill_opacity=0.3,
                    popup=f"Método {j+1}"
                ).add_to(m)

    return m

# Datos de ejemplo
points_list = [
    [
        [37.7749, -122.4194],
        [37.7849, -122.4094],
        [37.7649, -122.4294],
        [37.7849, -122.4194],
        [37.7749, -122.4294]
    ],
    [
        [37.7549, -122.4394],
        [37.7449, -122.4494],
        [37.7249, -122.4594],
        [37.7649, -122.4694],
        [37.7849, -122.4794]
    ]
]

# Generar mapa
m = generar_mapa_con_poligonos(points_list)

In [90]:
m